<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/Perceiver/Fine_tune_Perceiver_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

rm: cannot remove 'transformers': No such file or directory
Cloning into 'transformers'...
remote: Enumerating objects: 95370, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 95370 (delta 5), reused 15 (delta 5), pack-reused 95353
Receiving objects: 100% (95370/95370), 72.35 MiB | 28.46 MiB/s, done.
Resolving deltas: 100% (68666/68666), done.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 5.1 MB/s 
     |███

In [ ]:
!pip install -q datasets

     |████████████████████████████████| 298 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 55.2 MB/s 
     |████████████████████████████████| 132 kB 73.8 MB/s 
     |████████████████████████████████| 243 kB 72.4 MB/s 
     |████████████████████████████████| 160 kB 69.8 MB/s 
     |████████████████████████████████| 271 kB 48.6 MB/s 
     |████████████████████████████████| 192 kB 49.1 MB/s 


## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [ ]:
from datasets import load_dataset

train_ds, test_ds = load_dataset("imdb", split=['train[:10]+train[-10:]', 'test[:5]+test[-5:]'])

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

We create id2label and label2id mappings, which are handy at inference time.

In [ ]:
labels = train_ds.features['label'].names
print(labels)

['neg', 'pos']


In [ ]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
print(id2label)

{0: 'neg', 1: 'pos'}


Next, we prepare the data for the model using the tokenizer. 

In [ ]:
from transformers import PerceiverTokenizer

tokenizer = PerceiverTokenizer.from_pretrained("deepmind/language-perceiver")

train_ds = train_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                        batched=True)
test_ds = test_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                      batched=True)

Downloading:   0%|          | 0.00/823 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/911 [00:00<?, ?B/s]

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

We set the format to PyTorch tensors, and create familiar PyTorch dataloaders.

In [ ]:
train_ds.set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=4)

Here we verify some things (always important to check out your data!).

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

attention_mask torch.Size([4, 2048])
input_ids torch.Size([4, 2048])
label torch.Size([4])


In [ ]:
tokenizer.decode(batch['input_ids'][3])

"Very smart, sometimes shocking, I just love it. It shoved one more side of David's brilliant talent. He impressed me greatly! David is the best. The movie captivates your attention for every second.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [ ]:
batch['label']

tensor([0, 0, 0, 1])

## Define model

Next, we define our model, and put it on the GPU.

In [ ]:
from transformers import PerceiverForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PerceiverForSequenceClassification.from_pretrained("deepmind/language-perceiver",
                                                               num_labels=2,
                                                               id2label=id2label,
                                                               label2id=label2id)
model.to(device)

Downloading:   0%|          | 0.00/767M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepmind/language-perceiver were not used when initializing PerceiverForSequenceClassification: ['perceiver.decoder.decoding_cross_attention.attention.self.layernorm1.bias', 'perceiver.decoder.decoding_cross_attention.mlp.dense1.bias', 'perceiver.decoder.decoding_cross_attention.attention.output.dense.bias', 'perceiver.decoder.decoding_cross_attention.attention.self.key.weight', 'perceiver.decoder.decoding_cross_attention.attention.self.layernorm1.weight', 'perceiver.decoder.decoding_cross_attention.attention.self.query.bias', 'perceiver.decoder.decoding_cross_attention.attention.self.key.bias', 'perceiver.decoder.decoding_cross_attention.attention.self.layernorm2.bias', 'embedding_decoder.bias', 'perceiver.decoder.output_position_encodings.position_embeddings', 'perceiver.decoder.decoding_cross_attention.mlp.dense2.bias', 'perceiver.decoder.decoding_cross_attention.mlp.dense1.weight', 'perceiver.decoder.decoding_cross_attention.attention.output.

PerceiverForSequenceClassification(
  (perceiver): PerceiverModel(
    (input_preprocessor): PerceiverTextPreprocessor(
      (embeddings): Embedding(262, 768)
      (position_embeddings): Embedding(2048, 768)
    )
    (embeddings): PerceiverEmbeddings()
    (encoder): PerceiverEncoder(
      (cross_attention): PerceiverLayer(
        (attention): PerceiverAttention(
          (self): PerceiverSelfAttention(
            (layernorm1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
            (layernorm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (query): Linear(in_features=1280, out_features=256, bias=True)
            (key): Linear(in_features=768, out_features=256, bias=True)
            (value): Linear(in_features=768, out_features=1280, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): PerceiverSelfOutput(
            (dense): Linear(in_features=1280, out_features=1280, bias=True)
          )
        )


## Train the model

Here we train the model using native PyTorch.

In [ ]:
from transformers import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(20):  # loop over the dataset multiple times
    print("Epoch:", epoch)
    for batch in tqdm(train_dataloader):
         # get the inputs; 
         inputs = batch["input_ids"].to(device)
         attention_mask = batch["attention_mask"].to(device)
         labels = batch["label"].to(device)

         # zero the parameter gradients
         optimizer.zero_grad()

         # forward + backward + optimize
         outputs = model(inputs=inputs, attention_mask=attention_mask, labels=labels)
         loss = outputs.loss
         loss.backward()
         optimizer.step()

         # evaluate
         predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
         accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
         print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

Epoch: 0


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.8304584622383118, Accuracy: 0.25
Loss: 0.9324597120285034, Accuracy: 0.75
Loss: 2.6381170749664307, Accuracy: 0.0
Loss: 0.7451722025871277, Accuracy: 0.5
Loss: 0.7533012628555298, Accuracy: 0.5
Epoch: 1


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 1.2992808818817139, Accuracy: 0.25
Loss: 1.0224494934082031, Accuracy: 0.25
Loss: 0.7040843963623047, Accuracy: 0.25
Loss: 0.6907804012298584, Accuracy: 0.5
Loss: 0.7203900814056396, Accuracy: 0.25
Epoch: 2


...

Loss: 2.05873966217041, Accuracy: 0.25
Loss: 0.08333556354045868, Accuracy: 1.0
Loss: 1.191737174987793, Accuracy: 0.5
Loss: 1.0786267518997192, Accuracy: 0.5
Loss: 0.743228554725647, Accuracy: 0.5
Epoch: 18


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.5613749027252197, Accuracy: 0.75
Loss: 0.3839970827102661, Accuracy: 0.75
Loss: 1.4717833995819092, Accuracy: 0.25
Loss: 0.9521416425704956, Accuracy: 0.25
Loss: 0.40143632888793945, Accuracy: 0.75
Epoch: 19


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.640500009059906, Accuracy: 0.75
Loss: 0.34529292583465576, Accuracy: 1.0
Loss: 0.16599130630493164, Accuracy: 1.0
Loss: 0.49584537744522095, Accuracy: 0.75
Loss: 0.42844119668006897, Accuracy: 0.75


## Evaluate the model

Finally, we evaluate the model on the test set. We use the Datasets library to compute the accuracy.

In [ ]:
from tqdm.notebook import tqdm
from datasets import load_metric

accuracy = load_metric("accuracy")

model.eval()
for batch in tqdm(test_dataloader):
      # get the inputs; 
      inputs = batch["input_ids"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      labels = batch["label"].to(device)

      # forward pass
      outputs = model(inputs=inputs, attention_mask=attention_mask)
      logits = outputs.logits 
      predictions = logits.argmax(-1).cpu().detach().numpy()
      references = batch["label"].numpy()
      accuracy.add_batch(predictions=predictions, references=references)

final_score = accuracy.compute()
print("Accuracy on test set:", final_score)

## Inference

In [ ]:
text = "I loved this movie, it's super good."

input_ids = tokenizer(text, return_tensors="pt").input_ids

# forward pass
outputs = model(inputs=input_ids.to(device))
logits = outputs.logits 
predicted_class_idx = logits.argmax(-1).item()

print("Predicted:", model.config.id2label[predicted_class_idx])